# Find Optimal PSF
===========================

- creation 29/06/2016
- author Sylvie Dagoret-Campagne



Find HD163466 with grating

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [2]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [3]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [4]:
from photutils.background import Background2D

In [5]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [6]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-06-29 18:33:48.084078
fits date format for the analysis :  2016-06-29T18:33:48


## Definitions and Constants

In [7]:
object_name='HD163466_grating'

### input files

In [8]:
path='./HD163466_grating'
rootfilename='AssScImHD163466_grating_' 
NumStart=1
NumStop=25

### output file (table)

In [9]:
outputtablefile='HD163466_grating_1-25_TablePSF.fits'

### make the filelist

In [10]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [11]:
filelist

['./HD163466_grating/AssScImHD163466_grating_1.fits',
 './HD163466_grating/AssScImHD163466_grating_2.fits',
 './HD163466_grating/AssScImHD163466_grating_3.fits',
 './HD163466_grating/AssScImHD163466_grating_4.fits',
 './HD163466_grating/AssScImHD163466_grating_5.fits',
 './HD163466_grating/AssScImHD163466_grating_6.fits',
 './HD163466_grating/AssScImHD163466_grating_7.fits',
 './HD163466_grating/AssScImHD163466_grating_8.fits',
 './HD163466_grating/AssScImHD163466_grating_9.fits',
 './HD163466_grating/AssScImHD163466_grating_10.fits',
 './HD163466_grating/AssScImHD163466_grating_11.fits',
 './HD163466_grating/AssScImHD163466_grating_12.fits',
 './HD163466_grating/AssScImHD163466_grating_13.fits',
 './HD163466_grating/AssScImHD163466_grating_14.fits',
 './HD163466_grating/AssScImHD163466_grating_15.fits',
 './HD163466_grating/AssScImHD163466_grating_16.fits',
 './HD163466_grating/AssScImHD163466_grating_17.fits',
 './HD163466_grating/AssScImHD163466_grating_18.fits',
 './HD163466_gratin

## Read Input files

In [12]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD163466_grating/AssScImHD163466_grating_1.fits
./HD163466_grating/AssScImHD163466_grating_2.fits
./HD163466_grating/AssScImHD163466_grating_3.fits
./HD163466_grating/AssScImHD163466_grating_4.fits
./HD163466_grating/AssScImHD163466_grating_5.fits
./HD163466_grating/AssScImHD163466_grating_6.fits
./HD163466_grating/AssScImHD163466_grating_7.fits
./HD163466_grating/AssScImHD163466_grating_8.fits
./HD163466_grating/AssScImHD163466_grating_9.fits
./HD163466_grating/AssScImHD163466_grating_10.fits
./HD163466_grating/AssScImHD163466_grating_11.fits
./HD163466_grating/AssScImHD163466_grating_12.fits
./HD163466_grating/AssScImHD163466_grating_13.fits
./HD163466_grating/AssScImHD163466_grating_14.fits
./HD163466_grating/AssScImHD163466_grating_15.fits
./HD163466_grating/AssScImHD163466_grating_16.fits
./HD163466_grating/AssScImHD163466_grating_17.fits
./HD163466_grating/AssScImHD163466_grating_18.fits
./HD163466_grating/AssScImHD163466_grating_19.fits
./HD163466_grating/AssScImHD163466_grati

In [ ]:
#basefile_list

## For control

uncomment for control

In [14]:
index=0

In [15]:
#print exposures_list[index]

In [16]:
#header_list[index]

In [17]:
#plt.imshow(data_list[index])

In [18]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [19]:
#data_list[index].data-bkg.background

## Background subtraction

In [20]:
correctedimage_list = []

In [21]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [22]:
NBIMAGES=len(correctedimage_list)

In [23]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [24]:
image_psf=np.zeros((NBIMAGES,6))

In [25]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [28]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 64
average prf(x) = 3.27 +/- 0.26 pixels ==> psf  3.09 +/- 0.23 arcsec 
average prf(y) = 3.89 +/- 0.63 pixels ==> psf  3.67 +/- 0.57 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 1 ==> NBSTARS = 77
average prf(x) = 3.24 +/- 0.26 pixels ==> psf  3.05 +/- 0.23 arcsec 
average prf(y) = 3.69 +/- 0.71 pixels ==> psf  3.48 +/- 0.64 arcsec 
image 2 ==> NBSTARS = 62
average prf(x) = 3.32 +/- 0.35 pixels ==> psf  3.13 +/- 0.32 arcsec 
average prf(y) = 3.75 +/- 0.64 pixels ==> psf  3.54 +/- 0.58 arcsec 
image 3 ==> NBSTARS = 52
average prf(x) = 3.41 +/- 0.28 pixels ==> psf  3.22 +/- 0.25 arcsec 
average prf(y) = 3.94 +/- 0.64 pixels ==> psf  3.72 +/- 0.58 arcsec 
image 4 ==> NBSTARS = 58
average prf(x) = 3.31 +/- 0.22 pixels ==> psf  3.13 +/- 0.20 arcsec 
average prf(y) = 3.75 +/- 0.66 pixels ==> psf  3.54 +/- 0.59 arcsec 
image 5 ==> NBSTARS = 37
average prf(x) = 3.42 +/- 0.12 pixels ==> psf  3.23 +/- 0.11 arcsec 
average prf(y) = 3.80 +/- 0.50 pixels ==> psf  3.59 +/- 0.45 arcsec 
image 6 ==> NBSTARS = 27
average prf(x) = 3.27 +/- 0.13 pixels ==> psf  3.08 +/- 0.12 arcsec 
average prf(y) = 3.90 +/- 0.51 pixels ==> psf  3.69 +/- 0.46 arcsec 
image 7 ==> NBSTARS = 

## Create astropy table

In [29]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [30]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,64,3.27269668874,3.88553341132,0.255804340104,0.630429467266
1,77,3.23553358593,3.68556328674,0.257597799449,0.710772994991
2,62,3.32012000341,3.75058997275,0.349992606517,0.639362413853
3,52,3.40832433709,3.94203727252,0.278009048375,0.638777321358
4,58,3.31063509983,3.74642615098,0.223678199732,0.655809560281
5,37,3.41812953386,3.80001190974,0.122160814787,0.498240852554
6,27,3.26659530388,3.90464523342,0.127364793996,0.506540068331
7,35,3.39261307653,3.96786115165,0.274179753652,0.516360179579
8,44,3.19352421633,3.71210090293,0.207016221129,0.591840708818


In [31]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [32]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [33]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str31,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD163466_grating_1.fits,2016-05-12T07:44:43.031,5.0,64,3.27269668874,3.88553341132,0.255804340104,0.630429467266
1,AssScImHD163466_grating_2.fits,2016-05-12T07:45:25.729,5.0,77,3.23553358593,3.68556328674,0.257597799449,0.710772994991
2,AssScImHD163466_grating_3.fits,2016-05-12T07:45:39.969,5.0,62,3.32012000341,3.75058997275,0.349992606517,0.639362413853
3,AssScImHD163466_grating_4.fits,2016-05-12T07:45:53.737,5.0,52,3.40832433709,3.94203727252,0.278009048375,0.638777321358
4,AssScImHD163466_grating_5.fits,2016-05-12T07:46:08.135,5.0,58,3.31063509983,3.74642615098,0.223678199732,0.655809560281
5,AssScImHD163466_grating_6.fits,2016-05-12T07:46:21.770,5.0,37,3.41812953386,3.80001190974,0.122160814787,0.498240852554
6,AssScImHD163466_grating_7.fits,2016-05-12T07:47:45.901,5.0,27,3.26659530388,3.90464523342,0.127364793996,0.506540068331
7,AssScImHD163466_grating_8.fits,2016-05-12T07:47:54.975,5.0,35,3.39261307653,3.96786115165,0.274179753652,0.516360179579
8,AssScImHD163466_grating_9.fits,2016-05-12T07:48:04.034,5.0,44,3.19352421633,3.71210090293,0.207016221129,0.591840708818


## Write output file

In [34]:
t.write(outputtablefile,format='fits')

## Pandas

In [35]:
df=t.to_pandas()

In [36]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,25.000000,25.0,25.000000,25.000000,25.000000,25.000000,25.000000
mean,12.000000,3.4,31.960000,3.380461,3.818436,0.162651,0.408679
std,7.359801,2.0,20.210311,0.113820,0.103179,0.096924,0.226536
min,0.000000,1.0,9.000000,3.193524,3.642986,0.048746,0.109130
25%,6.000000,1.0,12.000000,3.298594,3.746426,0.073465,0.149382
50%,12.000000,5.0,35.000000,3.387251,3.813110,0.132763,0.506540
75%,18.000000,5.0,42.000000,3.448174,3.904645,0.255804,0.596840
max,24.000000,5.0,77.000000,3.654467,3.982965,0.349993,0.710773
